## AI Search Evals

- https://github.com/braintrustdata/braintrust-cookbook/blob/main/examples/AISearch/ai_search_evals.ipynb

 가이드는 Braintrust의 평가 워크플로우를 활용하여 AI 기반 검색 바를 개발한 과정을 보여줍니다.   


- 특정 실험을 찾으려면 표준 SQL 구문을 사용하여 검색창에 필터 및 정렬 쿼리를 입력할 수 있음 
- 하지만 SQL은 까다로울 수 있음. 
- 큰따옴표 대신 작은따옴표를 사용하거나, 잘못된 JSON 추출 구문, 오타와 같은 구문 오류가 쉽게 발생함. 
- 사용자는 직관적인 검색어(예: git 커밋 2a43fd1에서 실행된 실험 또는 점수 0.5 미만)를 입력하기만 하면 해당 SQL 쿼리가 자동으로 표시되기를 원할 것임. 
- Braintrust의 eval 프레임워크를 활용해 AI로 이를 구현해 보겠음.


In [1]:
from json import load
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import braintrust
import openai

In [3]:
PROJECT_NAME = "AI Search Cookbook"

# We use the Braintrust proxy here to get access to caching, but this is totally optional!
openai_opts = dict(
    base_url="https://api.braintrust.dev/v1/proxy",
    api_key=os.environ.get("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY"),
)
client = braintrust.wrap_openai(openai.AsyncOpenAI(default_headers={"x-bt-use-cache": "always"}, **openai_opts))

braintrust.login(api_key=os.environ.get("BRAINTRUST_API_KEY", "YOUR_BRAINTRUST_API_KEY"))
dataset = braintrust.init_dataset(PROJECT_NAME, "AI Search Cookbook Data", use_output=False)

### Load data and Render the templates

검색 쿼리를 GPT가 번역하도록 요청할 때는 여러 출력 옵션을 고려해야 합니다: 
(1) SQL 필터, (2) SQL 정렬, (3) 위 둘 다, (4) 번역 실패(예: 의미 없는 사용자 입력). 
각기 다른 시나리오를 견고하게 처리하기 위해 함수 호출(function calling)을 사용하며, 출력 형식은 다음과 같습니다:

* **match**: 모델이 검색을 유효한 SQL 필터/정렬로 번역할 수 있었는지 여부.
* **filter**: `WHERE` 절.
* **sort**: `ORDER BY` 절.
* **explanation**: 위 선택에 대한 설명 
  
이런 형태가 디버깅에 유용 


In [4]:
import dataclasses
from typing import Literal, Optional, Union

from pydantic import BaseModel, Field, create_model

In [5]:
@dataclasses.dataclass
class FunctionCallOutput:
    match: Optional[bool] = None
    filter: Optional[str] = None
    sort: Optional[str] = None
    explanation: Optional[str] = None
    error: Optional[str] = None

In [6]:
class Match(BaseModel):
    type: Literal["MATCH"] = "MATCH"
    explanation: str = Field(
        ..., description="Explanation of why I called the MATCH function"
    )


class SQL(BaseModel):
    type: Literal["SQL"] = "SQL"
    filter: Optional[str] = Field(..., description="SQL filter clause")
    sort: Optional[str] = Field(..., description="SQL sort clause")
    explanation: str = Field(
        ...,
        description="Explanation of why I called the SQL function and how I chose the filter and/or sort clauses",
    )

In [7]:
class Query(BaseModel):
    value: Union[Match, SQL] = Field(
        ...,
    )


def function_choices():
    return [
        {
            "name": "QUERY",
            "description": "Break down the query either into a MATCH or SQL call",
            "parameters": Query.model_json_schema(),
        },
    ]

wip